In [5]:
import ccxt
import pandas as pd
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
from binance.client import Client
import requests
import ta
import talib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import RobustScaler
from keras.models import load_model
import streamlit as st

# Definir las funciones necesarias
def fetch_latest_data_kraken(symbol, limit):
    exchange = ccxt.kraken()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1h', limit=limit)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    return data

def initialize_support_resistance(df):
    df['support'] = df['low'].rolling(window=30).min()
    df['resistance'] = df['high'].rolling(window=30).max()
    df['dynamic_support'] = df['low'].rolling(window=5).min()
    df['dynamic_resistance'] = df['high'].rolling(window=5).max()
    return df

def preprocess_data(df):
    df['sma_7'] = df['close'].rolling(window=7).mean()
    df['ema_7'] = df['close'].ewm(span=7, adjust=False).mean()
    df['sma_14'] = df['close'].rolling(window=14).mean()
    df['ema_14'] = df['close'].ewm(span=14, adjust=False).mean()
    df['sma_21'] = df['close'].rolling(window=21).mean()
    df['ema_21'] = df['close'].ewm(span=21, adjust=False).mean()
    df['sma_28'] = df['close'].rolling(window=28).mean()
    df['ema_28'] = df['close'].ewm(span=28, adjust=False).mean()
    df['sma_50'] = df['close'].rolling(window=50).mean()
    df['ema_50'] = df['close'].ewm(span=50, adjust=False).mean()
    df['sma_100'] = df['close'].rolling(window=100).mean()
    df['ema_100'] = df['close'].ewm(span=100, adjust=False).mean()
    df['sma_200'] = df['close'].rolling(window=200).mean()
    df['ema_200'] = df['close'].ewm(span=200, adjust=False).mean()
    df['rsi_14'] = talib.RSI(df['close'], timeperiod=14)
    df['macd'], df['macd_signal'], df['macd_diff'] = talib.MACD(df['close'])
    df['willr'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=14)
    df['atr_14'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['adx'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['cci'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    df['ichimoku_a'] = (talib.MIN(df['low'], timeperiod=9) + talib.MAX(df['high'], timeperiod=9)) / 2
    df['ichimoku_b'] = (talib.MIN(df['low'], timeperiod=26) + talib.MAX(df['high'], timeperiod=26)) / 2
    df['momentum_10'] = talib.MOM(df['close'], timeperiod=10)
    df['momentum_14'] = talib.MOM(df['close'], timeperiod=14)
    df['momentum_20'] = talib.MOM(df['close'], timeperiod=20)
    df['momentum_30'] = talib.MOM(df['close'], timeperiod=30)
    df['keltner_hband'] = talib.MA((df['high'] + df['low'] + df['close']) / 3 + 2 * df['atr_14'], timeperiod=10)
    df['keltner_lband'] = talib.MA((df['high'] + df['low'] + df['close']) / 3 - 2 * df['atr_14'], timeperiod=10)
    df['doji'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
    df['engulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
    df['hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['inverted_hammer'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
    df['hanging_man'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
    df['morning_star'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
    df['evening_star'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'])
    df['morning_doji_star'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
    df['evening_doji_star'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
    df['piercing_line'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
    df['dark_cloud_cover'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'])
    df['three_white_soldiers'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
    df['three_black_crows'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
    df['three_inside_up_down'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
    df['three_outside_up_down'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
    df['three_stars_in_the_south'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
    df['three_advancing_white_soldiers'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])

    df = df.fillna(0)
    return df

def create_sequences(X, y, seq_length):
    xs, ys = [], []
    for i in range(len(X) - seq_length):
        xs.append(X[i:i + seq_length])
        ys.append(y[i + seq_length])
    return np.array(xs), np.array(ys)

def trading_strategy(df, mape_threshold, buy_threshold, sell_threshold, stop_loss_percentage):
    signals = []
    confidences = []
    stop_losses = []
    for i in range(len(df) - 1):
        if df['MAPE'].iloc[i] > mape_threshold:
            signals.append('No hacer nada')
            confidences.append(0)
            stop_losses.append(np.nan)
        else:
            confidence = abs(df['corrected_prediction'].iloc[i + 1] - df['close'].iloc[i]) / df['close'].iloc[i]
            confidences.append(confidence)
            if df['corrected_prediction'].iloc[i + 1] > df['close'].iloc[i] + buy_threshold:
                signals.append('Comprar')
                stop_loss = df['close'].iloc[i] * (1 - stop_loss_percentage)
                stop_losses.append(stop_loss)
            elif df['corrected_prediction'].iloc[i + 1] < df['close'].iloc[i] - sell_threshold:
                signals.append('Vender')
                stop_losses.append(np.nan)
            else:
                signals.append('Mantener')
                stop_losses.append(np.nan)
    signals.append('Mantener')
    confidences.append(np.nan)
    stop_losses.append(np.nan)
    return signals, confidences, stop_losses

def evaluate_signal_accuracy(df):
    accuracies = []
    for i in range(len(df) - 1):
        if df['signal'].iloc[i] == 'Comprar':
            accuracies.append(df['next_close'].iloc[i] > df['close'].iloc[i])
        elif df['signal'].iloc[i] == 'Vender':
            accuracies.append(df['next_close'].iloc[i] < df['close'].iloc[i])
        elif df['signal'].iloc[i] == 'Mantener':
            accuracies.append(abs(df['next_close'].iloc[i] - df['close'].iloc[i]) < 0.01 * df['close'].iloc[i])
        else:
            accuracies.append(np.nan)
    accuracies.append(np.nan)
    return accuracies

def calculate_last_10_accuracy(df):
    df_last_10 = df.iloc[-11:-1]
    total_signals = df_last_10['accuracy'].notna().sum()
    correct_signals = (df_last_10['accuracy'] == True).sum()
    return correct_signals / total_signals * 100 if total_signals > 0 else 0

# Interfaz de Streamlit
def main():
    st.title('Aplicación de Trading para Criptomonedas')
    st.sidebar.header('Parámetros')
    
    crypto_symbol = st.sidebar.selectbox('Selecciona la criptomoneda', ['BTC/USDT', 'ETH/USDT', 'XRP/USDT', 'LTC/USDT'])
    limit = st.sidebar.number_input('Número de registros (60000 recomendado)', min_value=1000, max_value=100000, value=60000)
    mape_threshold = st.sidebar.slider('Umbral de MAPE (%)', 0, 10, 3)
    buy_threshold = st.sidebar.slider('Umbral de compra (%)', 0.0, 5.0, 0.01)
    sell_threshold = st.sidebar.slider('Umbral de venta (%)', 0.0, 5.0, 0.01)
    stop_loss_percentage = st.sidebar.slider('Stop-loss (%)', 0.0, 10.0, 2.0)
    seq_length = st.sidebar.slider('Longitud de la secuencia', 1, 100, 50)

    if st.sidebar.button('Ejecutar Predicción'):
        df = fetch_latest_data_kraken(crypto_symbol, limit)
        df = initialize_support_resistance(df)
        df = preprocess_data(df)

        df['next_close'] = df['close'].shift(-1)

        features = df.drop(columns=['next_close'])
        target = df['next_close']

        scaler = RobustScaler()
        features_scaled = scaler.fit_transform(features)
        target_scaled = scaler.fit_transform(target.values.reshape(-1, 1))

        X, y = create_sequences(features_scaled, target_scaled, seq_length)

        model_path = f'../models/best_model_{crypto_symbol.replace("/", "")}.keras'
        model = load_model(model_path)

        if X.shape[1:] != model.input_shape[1:]:
            st.error(f"Shape mismatch: X shape is {X.shape[1:]}, model expects {model.input_shape[1:]}")
            return

        predictions = []
        for i in range(11):
            input_data = X[-(11 - i)].reshape(1, seq_length, features_scaled.shape[1])
            pred = model.predict(input_data)
            predictions.append(scaler.inverse_transform(pred)[0][0])

        df['predictions'] = np.nan
        df.iloc[-11:, df.columns.get_loc('predictions')] = predictions

        actual = df['next_close'].iloc[-11:]
        predicted = df['predictions'].iloc[-11:]
        mape = np.mean(np.abs((actual - predicted) / actual)) * 100

        df['corrected_prediction'] = df['predictions'] * (1 + mape / 100)
        df['MAPE'] = mape
        df['signal'], df['confidence'], df['stop_loss'] = trading_strategy(df, mape_threshold, buy_threshold, sell_threshold, stop_loss_percentage)
        df['accuracy'] = evaluate_signal_accuracy(df)

        last_10_accuracy = calculate_last_10_accuracy(df)

        df.reset_index(inplace=True)

        st.write(df[['timestamp', 'close', 'next_close', 'predictions', 'corrected_prediction', 'signal', 'confidence', 'stop_loss', 'accuracy']].tail(11))

        st.write("Señales de Trading:")
        for index, row in df[['timestamp', 'signal', 'confidence', 'stop_loss', 'accuracy']].tail(11).iterrows():
            st.write(f"Timestamp: {row['timestamp']}, Señal: {row['signal']}, Confianza: {row['confidence']:.2f}, Stop-Loss: {row['stop_loss']}, Exactitud: {row['accuracy']}")

        st.write(f"Precisión global de las señales en las últimas 10 sesiones: {last_10_accuracy:.2f}%")
        st.write(f"Error Absoluto Medio Porcentual (MAPE): {mape:.2f}%")

        current_time = datetime.now()
        if current_time.minute > 20:
            st.warning("Faltan menos de 40 minutos para el cierre de la hora actual. Es mejor pedir información en la siguiente hora.")

if __name__ == "__main__":
    main()
